This notebook will scrape and clean 4 years of movie data from BoxOfficeMojo.com.
We want to determine which features about a movie will result in a large total gross
despite a slow start (low opening gross)

In [123]:
import requests
import re
import pandas as pd
import lxml

from bs4 import BeautifulSoup
import dateutil.parser
import pickle
import string
import sys
import numpy as np

### Scrape Data From BoxOfficeMojo

In [108]:
#Scrape 4 years worth of movie gross data from BoxOfficeMojo
#Collect movie, opening and total gross figures, opening and total theater count,
#opening and closing dates, director, genre, production budget, rating and runtime data
year_list = ['2012', '2013', '2014', '2015']
movie_list = []
patternUrl = '"(.*?)"'
for year in year_list:
    number = 1
    while number < 8:
        url = 'http://www.boxofficemojo.com/yearly/chart/?page=' + str(number) + \
            '&view=releasedate&view2=domestic&yr=' + year + '&p=.htm'
            
        # Get charts of movie gross data
        use_df = pd.read_html(url)[5]
        use_df.columns = ['Rank', 'Title', 'Studio', 'Total Gross', 'Total Theaters', 'Opening Gross',
                         'Opening Theaters', 'Open Date', 'Close Date']
        use_df = use_df.drop(use_df.index[[0,1]])
        use_df = use_df.drop(use_df.index[-4:])
        use_df['Year'] = year
        
        # Get links for additional data
        id_list = []
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "html.parser")
        a_list = soup.find_all('table')[6].find_all('a', href=True)
        k = 9
        
        director_list = []
        rating_list = []
        prodbud_list = []
        runtime_list = []
        genre_list = []
        while k < len(a_list):
            link = re.findall(patternUrl, str(a_list[k]))
            id_list.append(link[0])
            sys.stdout.write("\r" + 'Year: ' + year + ' Page: ' + str(number) + ' URL count: ' + str(k))
            k = k + 3
            
            # Get additional pertinent data
            movie_url = 'http://www.boxofficemojo.com' + link[0]
            response = requests.get(movie_url)
            page = response.text
            soup = BeautifulSoup(page, "html.parser")
            links = soup.find_all('a')
            i = 0
            
            try:
                for link in links:
                    if 'Director' in link.text:
                        dir_index = i
                    if 'Writer' in link.text:
                        write_index = i
                    i += 1
                sub_directors = ''
                for m in range(dir_index + 1, write_index):
                    sub_directors = sub_directors + links[m].text + ' '
                director_list.append(sub_directors)
            except:
                director_list.append('N/A')

            try:
                holderFrame = pd.read_html(movie_url)
                genFrame = holderFrame[5]
                rating_data = genFrame[0][3]
                prodbud_data = genFrame[1][3]
                runtime_data = genFrame[1][2]
                genre_data = genFrame[0][2]
            except:
                rating_data = 'N/A'
                prodbud_data = 'N/A'
                runtime_data = 'N/A'
                genre_data = 'N/A'
            
            rating_list.append(rating_data)
            prodbud_list.append(prodbud_data)
            runtime_list.append(runtime_data)
            genre_list.append(genre_data)
            
        use_df['partial link'] = id_list
        use_df['Director(s)'] = director_list
        use_df['Rating'] = rating_list
        use_df['Production Budget'] = prodbud_list
        use_df['Runtime (hrs.)'] = runtime_list
        use_df['Genre'] = genre_list        
        
        movie_list.append(use_df)
        if year != '2014' and number == 7:
            number += 1
        number += 1

movie_df = pd.concat(movie_list, ignore_index=True)

Year: 2015 Page: 7 URL count: 306

In [110]:
#Clean unneeded characters from some columns
movie_df['Rating'] = movie_df['Rating'].map(lambda x: x.strip('MPAA Rating:'))
movie_df['Runtime'] = movie_df['Runtime'].map(lambda x: x.strip('Runtime:'))
movie_df['Genre'] = movie_df['Genre'].map(lambda x: x.strip('Genre:'))
movie_df['Production Budget'] = movie_df['Production Budget'].map(lambda x: x.strip('Production Budget:$'))
movie_df['Total Gross'] = movie_df['Total Gross'].map(lambda x: x.strip('$'))
movie_df['Total Gross'] = movie_df['Total Gross'].map(lambda x: x.replace(',', ''))
movie_df['Total Gross'] = movie_df['Total Gross'].map(lambda x: float(x))
movie_df['Opening Gross'] = movie_df['Opening Gross'].map(lambda x: str(x))
movie_df['Opening Gross'] = movie_df['Opening Gross'].map(lambda x: x.strip('$'))
movie_df['Opening Gross'] = movie_df['Opening Gross'].map(lambda x: float(x))

In [171]:
#Convert Production Budget values to numbers
new_prodbud_list = []
for x in movie_df['Production Budget']:
    p += 1
    if 'mill' not in x and 'N/A' not in x:
        x = x.replace(',', '')
        x = float(x)
        new_prodbud_list.append(x)
    elif 'mill' in x:
        x = x.strip(' mill')
        x = x + '000000'
        x = float(x)
        new_prodbud_list.append(x)
    elif 'N/A' in x:
        x = np.nan
        new_prodbud_list.append(x)
movie_df['Production Budget'] = new_prodbud_list

[220000000.0,
 250000000.0,
 78000000.0,
 200000000.0,
 nan,
 120000000.0,
 230000000.0,
 185000000.0,
 50000000.0,
 145000000.0,
 70000000.0,
 165000000.0,
 65000000.0,
 225000000.0,
 100000000.0,
 95000000.0,
 170000000.0,
 61000000.0,
 85000000.0,
 45000000.0,
 42000000.0,
 44.5,
 21000000.0,
 130000000.0,
 85000000.0,
 30000000.0,
 120000000.0,
 7000000.0,
 125000000.0,
 79000000.0,
 145000000.0,
 40000000.0,
 31000000.0,
 12000000.0,
 nan,
 100000000.0,
 150000000.0,
 60000000.0,
 150000000.0,
 25000000.0,
 250000000.0,
 12000000.0,
 35000000.0,
 25000000.0,
 30000000.0,
 20000000.0,
 209000000.0,
 17000000.0,
 85000000.0,
 12000000.0,
 nan,
 70000000.0,
 nan,
 65000000.0,
 125000000.0,
 18000000.0,
 50000000.0,
 60000000.0,
 65000000.0,
 nan,
 nan,
 5000000.0,
 1000000.0,
 nan,
 57000000.0,
 25000000.0,
 58000000.0,
 14000000.0,
 22000000.0,
 3000000.0,
 nan,
 45000000.0,
 10000000.0,
 16000000.0,
 nan,
 65000000.0,
 30000000.0,
 nan,
 65000000.0,
 nan,
 40000000.0,
 nan,
 700000

In [181]:
#Convert runtime values to times
new_runtime_list = []
p = 0
for x in movie_df['Runtime']:
    p += 1
    x = x.split(' ')
    if len(x) == 5:
        x = round(float(x[1]) + float(x[3]) / 60, ndigits=2)
        new_runtime_list.append(x)
    else:
        x = np.nan
        new_runtime_list.append(x)
    #if p == 10:
    #    break
movie_df['Runtime'] = new_runtime_list

[2.37,
 2.75,
 2.37,
 2.38,
 2.77,
 1.93,
 2.27,
 1.67,
 1.77,
 1.55,
 1.58,
 1.8,
 2.42,
 1.77,
 2.75,
 1.57,
 2.12,
 2.62,
 1.52,
 1.52,
 1.82,
 2.0,
 2.03,
 2.07,
 1.95,
 1.73,
 2.1,
 1.83,
 2.25,
 1.57,
 1.62,
 2.62,
 2.32,
 2.03,
 1.62,
 1.7,
 1.65,
 2.17,
 1.88,
 1.73,
 2.2,
 1.85,
 2.22,
 1.83,
 1.97,
 1.9,
 2.18,
 1.87,
 1.77,
 1.38,
 2.02,
 1.47,
 1.68,
 1.38,
 2.02,
 3.23,
 1.88,
 1.55,
 1.62,
 1.47,
 1.58,
 1.58,
 1.45,
 1.67,
 1.58,
 1.95,
 2.0,
 1.52,
 1.57,
 1.83,
 1.4,
 2.17,
 1.97,
 1.55,
 1.75,
 1.55,
 1.53,
 2.22,
 1.58,
 1.58,
 1.83,
 1.73,
 1.82,
 2.05,
 1.75,
 1.92,
 1.58,
 1.9,
 1.85,
 1.7,
 1.45,
 1.67,
 2.15,
 1.5,
 1.48,
 1.68,
 1.47,
 1.97,
 1.3,
 2.07,
 2.73,
 1.52,
 1.68,
 1.58,
 2.0,
 1.52,
 2.05,
 1.57,
 1.78,
 1.55,
 1.7,
 1.52,
 1.5,
 1.72,
 1.57,
 1.63,
 1.58,
 1.58,
 2.28,
 1.72,
 1.6,
 1.62,
 1.82,
 0.67,
 1.83,
 1.67,
 1.58,
 1.58,
 2.17,
 1.52,
 1.42,
 1.52,
 1.92,
 1.42,
 1.6,
 1.87,
 1.5,
 1.73,
 1.87,
 1.67,
 2.33,
 1.67,
 1.57,
 1.35,
 2.08,
 1.

In [187]:
movie_df

,Rank,Title,Studio,Total Gross,Total Theaters,Opening Gross,Opening Theaters,Open Date,Close Date,Year,partial link,Director(s),Rating,Production Budget,Runtime,Genre
0,1,Marvel's The Avengers,BV,623357910.0,4349,207438708.0,4349,5/4,10/4,2012,/movies/?id=avengers11.htm,Joss Whedon,G-13,220000000.0,2.37,Action / Adventu
1,2,The Dark Knight Rises,WB,448139099.0,4404,160887295.0,4404,7/20,12/13,2012,/movies/?id=batman3.htm,Christopher Nolan,G-13,250000000.0,2.75,Action Thrill
2,3,The Hunger Games,LGF,408010692.0,4137,152535747.0,4137,3/23,9/6,2012,/movies/?id=hungergames.htm,Gary Ross,G-13,78000000.0,2.37,Action / Adventu
3,4,Skyfall,Sony,304360277.0,3526,88364714.0,3505,11/9,3/10,2012,/movies/?id=bond23.htm,Sam Mendes,G-13,200000000.0,2.38,Actio
4,5,The Hobbit: An Unexpected Journey,WB (NL),303003568.0,4100,84617303.0,4045,12/14,4/25,2012,/movies/?id=hobbit.htm,Peter Jackson,G-13,NaN,2.77,Fantasy
5,6,The Twilight Saga: Breaking Dawn Part 2,LG/S,292324737.0,4070,141067634.0,4070,11/16,3/7,2012,/movies/?id=breakingdawn2.htm,Bill Condon,G-13,120000000.0,1.93,Romanc
6,7,The Amazing Spider-Man,Sony,262030663.0,4318,62004688.0,4318,7/3,10/14,2012,/movies/?id=spiderman4.htm,Marc Webb,G-13,230000000.0,2.27,Action / Adventu
7,8,Brave,BV,237283207.0,4164,66323594.0,4164,6/22,1/17,2012,/movies/?id=bearandthebow.htm,Mark Andrews Brenda Chapman,G,185000000.0,1.67,Animatio
8,9,Ted,Uni.,218815487.0,3303,54415205.0,3239,6/29,10/25,2012,/movies/?id=ted.htm,Seth MacFarlane,,50000000.0,1.77,Comedy
9,10,Madagascar 3: Europe's Most Wanted,P/DW,216391482.0,4263,60316738.0,4258,6/8,10/18,2012,/movies/?id=madagascar3.htm,Eric Darnell Tom McGrath Conrad Vernon,G,145000000.0,1.55,Animatio


In [191]:
#with open('rerun_jul2016_df.pkl', 'w') as picklefile:
#    pickle.dump(movie_df, picklefile)
movie_df.to_csv('rerun_jul2016_df.csv')

In [192]:
!ls

DataReading.py                      df2015data_2.pkl
MRegalla_Movie_Presentation.pdf     getdata.py
Movie.pptx                          infoList.pkl
Music_Futures_ideas                 luther02_challenges
OpenVsTotal_Analysis_MVP.ipynb      luther_challenges
OpenVsTotal_Analysis_Onward.ipynb   myfile.txt
OpeningVsTotal_GetData.ipynb        okdf.pkl
Opening_vs_Total_Get_Data_py3.ipynb opening vs gross
README.md                           pairprogram
Scrape_more_Stuff.ipynb             rerun_jul2016_df.csv
Untitled.ipynb                      rerun_jul2016_df.pkl
bigList.pkl                         totalFrame.pkl


In [103]:
dummies = pd.get_dummies(a_df)
dummies

,col 1_blue,col 1_green,col 1_red
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0


In [133]:
theater_series = movie_df['Total Theaters'].convert_objects(convert_numeric=True)
pd.cut(theater_series, 3, retbins=True)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


(0           (2936.333, 4404]
 1           (2936.333, 4404]
 2           (2936.333, 4404]
 3           (2936.333, 4404]
 4           (2936.333, 4404]
 5           (2936.333, 4404]
 6           (2936.333, 4404]
 7           (2936.333, 4404]
 8           (2936.333, 4404]
 9           (2936.333, 4404]
 10          (2936.333, 4404]
 11          (2936.333, 4404]
 12      (1468.667, 2936.333]
 13          (2936.333, 4404]
 14          (2936.333, 4404]
 15          (2936.333, 4404]
 16          (2936.333, 4404]
 17      (1468.667, 2936.333]
 18          (2936.333, 4404]
 19          (2936.333, 4404]
 20          (2936.333, 4404]
 21          (2936.333, 4404]
 22      (1468.667, 2936.333]
 23          (2936.333, 4404]
 24          (2936.333, 4404]
 25          (2936.333, 4404]
 26          (2936.333, 4404]
 27          (2936.333, 4404]
 28          (2936.333, 4404]
 29          (2936.333, 4404]
                 ...         
 2725      (-3.403, 1468.667]
 2726      (-3.403, 1468.667]
 2727     